In [11]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=d75e806edf37e43d1f87a6879f41535359211d9a4d48c7b19d9fda70aa1668dd
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=76f985c8521a49cd0e2d0405bbb496e87503d0197b2c9e3a67f77064af1c2e4a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [13]:
fashion_mnist = keras.datasets.fashion_mnist

In [14]:
(train_images, train_label), (test_images, test_label) = fashion_mnist.load_data()

In [15]:
train_images = train_images/255.0
test_images = test_images/255.0

In [16]:
train_images[0].shape

(28, 28)

In [17]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [18]:
train_images[0].shape

(28, 28, 1)

In [19]:
def build_model(hp):
  model = keras.Sequential([
          keras.layers.Conv2D(
              filters = hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
              kernel_size = hp.Choice('conv_1_kernel', values= [3,5]),
              activation = 'relu',
              input_shape = (28,28,1)
          ),
           keras.layers.Conv2D(
              filters = hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
              kernel_size = hp.Choice('conv_2_kernel', values= [3,5]),
              activation = 'relu',
          ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units = hp.Int('dense_1_units', min_value = 32, max_value= 128, step= 16),
              activation = 'relu'
          ),
          keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model


In [20]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [21]:
tuner_search = RandomSearch(build_model, objective="val_accuracy", max_trials=5, directory ="output", project_name = "Mnist Fashion")

In [22]:
tuner_search.search(train_images, train_label, epochs = 3, validation_split = 0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3864 - accuracy: 0.8607 - val_loss: 0.2876 - val_accuracy: 0.8957
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2508 - accuracy: 0.9076 - val_loss: 0.2723 - val_accuracy: 0.8988
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1903 - accuracy: 0.9281 - val_loss: 0.2486 - val_accuracy: 0.9142


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3785 - accuracy: 0.8638 - val_loss: 0.2941 - val_accuracy: 0.8910
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2410 - accuracy: 0.9097 - val_loss: 0.2533 - val_accuracy: 0.9082
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1805 - accuracy: 0.9319 - val_loss: 0.2575 - val_accuracy: 0.9105


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4274 - accuracy: 0.8469 - val_loss: 0.3007 - val_accuracy: 0.8872
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2796 - accuracy: 0.8967 - val_loss: 0.2749 - val_accuracy: 0.8993
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.2300 - accuracy: 0.9151 - val_loss: 0.2755 - val_accuracy: 0.9040


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5529 - accuracy: 0.8126 - val_loss: 0.4298 - val_accuracy: 0.8428
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3916 - accuracy: 0.8549 - val_loss: 0.3957 - val_accuracy: 0.8512
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3674 - accuracy: 0.8628 - val_loss: 0.3803 - val_accuracy: 0.8570


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4400 - accuracy: 0.8426 - val_loss: 0.3380 - val_accuracy: 0.8768
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3151 - accuracy: 0.8842 - val_loss: 0.3222 - val_accuracy: 0.8883
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2742 - accuracy: 0.8990 - val_loss: 0.3211 - val_accuracy: 0.8885


INFO:tensorflow:Oracle triggered exit


In [24]:
model = tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 112)       113008    
_________________________________________________________________
flatten (Flatten)            (None, 54208)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               6071408   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 6,188,458
Trainable params: 6,188,458
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(train_images, train_label, validation_split=0.1, epochs=10, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1419 - accuracy: 0.9472 - val_loss: 0.2548 - val_accuracy: 0.9148
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1063 - accuracy: 0.9604 - val_loss: 0.3036 - val_accuracy: 0.9047
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0755 - accuracy: 0.9718 - val_loss: 0.3217 - val_accuracy: 0.9083
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0598 - accuracy: 0.9773 - val_loss: 0.3714 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0409 - accuracy: 0.9845 - val_loss: 0.3946 - val_accuracy: 0.9075
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0395 - accuracy: 0.9861 - val_loss: 0.4451 - val_accuracy: 0.9105
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0283 - accuracy: 0.9900 - val_loss: 0.4758 - val_accuracy